In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.multioutput import RegressorChain
from lightgbm import LGBMRegressor
import optuna
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_percentage_error
import xgboost as xgb
from catboost import CatBoostRegressor
from sklearn.linear_model import LinearRegression
import numpy as np
from xgboost import XGBRegressor



from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

from sklearn.ensemble import StackingRegressor

import matplotlib.pyplot as plt

from scipy import stats
import numpy as np
from sklearn.metrics import mean_absolute_error

import warnings
warnings.filterwarnings("ignore")

/Users/donaldaadithiyan/Personal learn dev/YT-analytics/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
preprocessed_dataset = pd.read_csv("../data/pre-processed/preprocessed_youtube_data.csv")

In [3]:
# Convert to datetime if not already
preprocessed_dataset['published_time'] = pd.to_datetime(preprocessed_dataset['published_time'], format='%H:%M:%S', errors='coerce')

# Extract hour and minute
preprocessed_dataset['published_hour'] = preprocessed_dataset['published_time'].dt.hour
preprocessed_dataset['published_minute'] = preprocessed_dataset['published_time'].dt.minute

# Drop the original time column
preprocessed_dataset.drop(columns=['published_time'], inplace=True)

In [4]:
# Initialize label encoder
le = LabelEncoder()

# Apply on 'published_day_of_week'
preprocessed_dataset['published_day_of_week'] = le.fit_transform(preprocessed_dataset['published_day_of_week'])

In [5]:
missing_mask = preprocessed_dataset[['like_count_initial', 'like_count_final', 'view_count_initial', 'view_count_final']].isnull()

# Check rows where all four columns are NaN
rows_all_nan = missing_mask.all(axis=1)

print("Number of rows with all four columns NaN:", rows_all_nan.sum())

# Optionally, see those rows
print(preprocessed_dataset[rows_all_nan])

Number of rows with all four columns NaN: 11
     category_id  view_count_initial  like_count_initial  \
22           NaN                 NaN                 NaN   
28           NaN                 NaN                 NaN   
32           NaN                 NaN                 NaN   
61           NaN                 NaN                 NaN   
70           NaN                 NaN                 NaN   
83           NaN                 NaN                 NaN   
130          NaN                 NaN                 NaN   
137          NaN                 NaN                 NaN   
179          NaN                 NaN                 NaN   
239          NaN                 NaN                 NaN   
307          NaN                 NaN                 NaN   

     c_view_count_initial  c_subscriber_count_initial  view_count_final  \
22                    NaN                         NaN               NaN   
28                    NaN                         NaN               NaN   
32       

In [6]:
rows_any_nan = missing_mask.any(axis=1)
print("Rows with any of the four columns NaN:", rows_any_nan.sum())

Rows with any of the four columns NaN: 11


In [7]:
preprocessed_dataset = preprocessed_dataset.dropna(subset=[
    'like_count_initial',
    'like_count_final',
    'view_count_initial',
    'view_count_final'
])

# Define target columns
target_columns = [
    'like_count_initial',
    'like_count_final',
    'view_count_initial',
    'view_count_final'
]

In [8]:
# Choose columns to check for outliers, e.g. target columns
cols_to_check = target_columns  # or other numeric features

z_scores = np.abs(stats.zscore(preprocessed_dataset[cols_to_check]))
# Define threshold, e.g. 3 std deviations
threshold = 3
# Keep only rows where all z-scores are below threshold (no outlier)
non_outliers = (z_scores < threshold).all(axis=1)


print(f"Rows before outlier removal: {len(preprocessed_dataset)}")
preprocessed_dataset = preprocessed_dataset[non_outliers]
print(f"Rows after outlier removal: {len(preprocessed_dataset)}")

Rows before outlier removal: 339
Rows after outlier removal: 332


In [9]:
# Separate features and targets
X = preprocessed_dataset.drop(columns=target_columns)
y = preprocessed_dataset[target_columns]

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_valid, y_train, y_valid = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [10]:
# 2. Define objective function for Optuna
def objective(trial):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 100, 1000),
        "max_depth": trial.suggest_int("max_depth", 3, 12),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
        "gamma": trial.suggest_float("gamma", 0, 5),
        "reg_alpha": trial.suggest_float("reg_alpha", 0.0, 5.0),
        "reg_lambda": trial.suggest_float("reg_lambda", 0.0, 5.0),
        "random_state": 42,
        "n_jobs": -1,
    }

    xgb = XGBRegressor(**params)
    model = MultiOutputRegressor(xgb)
    model.fit(X_train, y_train)

    preds = model.predict(X_valid)
    return mean_absolute_error(y_valid, preds)

# 3. Run Optuna study
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=30, show_progress_bar=True)

# 4. Train final model with best params
best_params = study.best_trial.params
best_model = MultiOutputRegressor(XGBRegressor(**best_params))
best_model.fit(X_train, y_train)

# 5. Predict and evaluate
y_pred = best_model.predict(X_valid)
mae = mean_absolute_error(y_valid, y_pred)
print(f"✅ Best MAE: {mae:.4f}")

[I 2025-07-31 01:20:57,492] A new study created in memory with name: no-name-38c373fe-bd3a-4499-82d3-74352872e2a8
Best trial: 0. Best value: 346.324:   3%|▎         | 1/30 [00:01<00:33,  1.16s/it]

[I 2025-07-31 01:20:58,658] Trial 0 finished with value: 346.3236083984375 and parameters: {'n_estimators': 225, 'max_depth': 5, 'learning_rate': 0.21478573006084015, 'subsample': 0.511308659836726, 'colsample_bytree': 0.8097227415308779, 'gamma': 2.180531647816659, 'reg_alpha': 1.3862919949385129, 'reg_lambda': 1.1269866631288672}. Best is trial 0 with value: 346.3236083984375.


Best trial: 0. Best value: 346.324:   7%|▋         | 2/30 [00:02<00:38,  1.37s/it]

[I 2025-07-31 01:21:00,177] Trial 1 finished with value: 387.4968566894531 and parameters: {'n_estimators': 314, 'max_depth': 4, 'learning_rate': 0.24377870045550062, 'subsample': 0.5347982525137283, 'colsample_bytree': 0.755670105923198, 'gamma': 4.80483162760829, 'reg_alpha': 0.9458012962267115, 'reg_lambda': 1.2651234939230804}. Best is trial 0 with value: 346.3236083984375.


Best trial: 2. Best value: 247.6:  10%|█         | 3/30 [00:04<00:41,  1.52s/it]  

...

[I 2025-07-31 01:22:03,691] Trial 27 finished with value: 271.36785888671875 and parameters: {'n_estimators': 117, 'max_depth': 10, 'learning_rate': 0.2631319186260312, 'subsample': 0.8911753783354792, 'colsample_bytree': 0.8431459927862685, 'gamma': 1.8633093231524152, 'reg_alpha': 3.3197583216690627, 'reg_lambda': 0.4153460027304302}. Best is trial 22 with value: 225.4829559326172.


Best trial: 22. Best value: 225.483:  97%|█████████▋| 29/30 [01:07<00:01,  1.46s/it]

[I 2025-07-31 01:22:05,052] Trial 28 finished with value: 240.138671875 and parameters: {'n_estimators': 725, 'max_depth': 8, 'learning_rate': 0.2728163412496014, 'subsample': 0.9569953152994097, 'colsample_bytree': 0.9082846643247853, 'gamma': 3.033562363405807, 'reg_alpha': 3.1733297624483763, 'reg_lambda': 0.9153380647119655}. Best is trial 22 with value: 225.4829559326172.


Best trial: 22. Best value: 225.483: 100%|██████████| 30/30 [01:09<00:00,  2.32s/it]


[I 2025-07-31 01:22:07,039] Trial 29 finished with value: 293.67523193359375 and parameters: {'n_estimators': 920, 'max_depth': 5, 'learning_rate': 0.2199555373944075, 'subsample': 0.8057887903259322, 'colsample_bytree': 0.8409577529379317, 'gamma': 1.9545636006542102, 'reg_alpha': 3.50034595689173, 'reg_lambda': 1.5359002043762857}. Best is trial 22 with value: 225.4829559326172.
✅ Best MAE: 231.3416


In [11]:
# Convert to DataFrame for easier handling if needed
if not isinstance(y_valid, pd.DataFrame):
    y_valid = pd.DataFrame(y_valid, columns=target_columns)
if not isinstance(y_pred, pd.DataFrame):
    y_pred = pd.DataFrame(y_pred, columns=target_columns)

# MAE per target column
mae_per_column = {}
for col in target_columns:
    mae_per_column[col] = mean_absolute_error(y_valid[col], y_pred[col])

print("MAE per target column:")
for col, val in mae_per_column.items():
    print(f"{col}: {val:.4f}")

MAE per target column:
like_count_initial: 22.9707
like_count_final: 25.5946
view_count_initial: 368.0239
view_count_final: 508.7772


In [12]:
def catboost_objective(trial):
    params = {
        "iterations": trial.suggest_int("iterations", 100, 1000),
        "depth": trial.suggest_int("depth", 3, 12),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3),
        "random_strength": trial.suggest_float("random_strength", 1, 20),
        "bagging_temperature": trial.suggest_float("bagging_temperature", 0.0, 1.0),
        "l2_leaf_reg": trial.suggest_float("l2_leaf_reg", 1, 10),
        "border_count": trial.suggest_int("border_count", 32, 255),
        "random_seed": 42,
        "verbose": False,
    }

    catBoost = CatBoostRegressor(**params)
    model = MultiOutputRegressor(catBoost)
    model.fit(X_train, y_train)

    preds = model.predict(X_valid)
    return mean_absolute_error(y_valid, preds)

# 3. Run Optuna study
study = optuna.create_study(direction="minimize")
study.optimize(catboost_objective, n_trials=30, show_progress_bar=True)

# 4. Train final model with best params
best_params = study.best_trial.params
best_model = MultiOutputRegressor(CatBoostRegressor(**best_params))
best_model.fit(X_train, y_train)

# 5. Predict and evaluate
y_pred = best_model.predict(X_valid)
mae = mean_absolute_error(y_valid, y_pred)

# Convert y_valid and y_pred to DataFrames if they aren't already, using target column names
if not isinstance(y_valid, pd.DataFrame):
    y_valid = pd.DataFrame(y_valid, columns=target_columns)
if not isinstance(y_pred, pd.DataFrame):
    y_pred = pd.DataFrame(y_pred, columns=target_columns)

# Calculate MAE for each column
print(f"✅ Best Overall MAE: {mae:.4f}\n")
print("MAE per target column:")
for col in target_columns:
    col_mae = mean_absolute_error(y_valid[col], y_pred[col])
    print(f"{col}: {col_mae:.4f}")

[I 2025-07-31 01:22:08,762] A new study created in memory with name: no-name-c72c9740-4343-4bbd-8532-513dfa8d5848
Best trial: 0. Best value: 364.302:   3%|▎         | 1/30 [00:13<06:45, 13.98s/it]

[I 2025-07-31 01:22:22,745] Trial 0 finished with value: 364.30183787981565 and parameters: {'iterations': 975, 'depth': 11, 'learning_rate': 0.22251246190479093, 'random_strength': 11.553643808546326, 'bagging_temperature': 0.3471956234128152, 'l2_leaf_reg': 1.118308364617864, 'border_count': 206}. Best is trial 0 with value: 364.30183787981565.


Best trial: 1. Best value: 362.652:   7%|▋         | 2/30 [00:14<02:44,  5.88s/it]

[I 2025-07-31 01:22:22,951] Trial 1 finished with value: 362.6524491972532 and parameters: {'iterations': 316, 'depth': 4, 'learning_rate': 0.19458463560685976, 'random_strength': 9.035684951805415, 'bagging_temperature': 0.30719781558802794, 'l2_leaf_reg': 1.2007499024507604, 'border_count': 115}. Best is trial 1 with value: 362.6524491972532.


Best trial: 2. Best value: 284.228:  10%|█         | 3/30 [00:16<01:53,  4.20s/it]

...

[I 2025-07-31 01:24:33,682] Trial 27 finished with value: 303.5666933190365 and parameters: {'iterations': 202, 'depth': 9, 'learning_rate': 0.13702219518533024, 'random_strength': 7.560586583804403, 'bagging_temperature': 0.4954113647511142, 'l2_leaf_reg': 3.893892698877099, 'border_count': 202}. Best is trial 21 with value: 270.9412034848383.


Best trial: 28. Best value: 270.088:  97%|█████████▋| 29/30 [02:26<00:03,  3.45s/it]

[I 2025-07-31 01:24:35,391] Trial 28 finished with value: 270.08771222067685 and parameters: {'iterations': 734, 'depth': 7, 'learning_rate': 0.0955022089482313, 'random_strength': 3.2072320690955545, 'bagging_temperature': 0.8680869165561409, 'l2_leaf_reg': 6.00419266618186, 'border_count': 255}. Best is trial 28 with value: 270.08771222067685.


Best trial: 28. Best value: 270.088: 100%|██████████| 30/30 [02:28<00:00,  4.94s/it]


[I 2025-07-31 01:24:37,088] Trial 29 finished with value: 352.5372411251092 and parameters: {'iterations': 727, 'depth': 7, 'learning_rate': 0.10242119623518066, 'random_strength': 10.998728364181467, 'bagging_temperature': 0.896707183805163, 'l2_leaf_reg': 5.621250949230314, 'border_count': 255}. Best is trial 28 with value: 270.08771222067685.
0:	learn: 123.6543999	total: 555us	remaining: 407ms
1:	learn: 121.6598514	total: 1.11ms	remaining: 406ms
2:	learn: 119.9307325	total: 1.62ms	remaining: 395ms
3:	learn: 117.8492138	total: 2.13ms	remaining: 389ms
4:	learn: 116.3616394	total: 3.6ms	remaining: 525ms
5:	learn: 114.7978239	total: 4.18ms	remaining: 508ms
6:	learn: 113.3952779	total: 4.67ms	remaining: 485ms
7:	learn: 111.2856447	total: 5.18ms	remaining: 470ms
8:	learn: 109.4341943	total: 5.69ms	remaining: 458ms
9:	learn: 108.1197030	total: 6.25ms	remaining: 453ms
10:	learn: 106.5484863	total: 6.73ms	remaining: 442ms
11:	learn: 105.6388313	total: 7.15ms	remaining: 430ms
12:	learn: 104.7

In [13]:
def lightGBM_objective(trial):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 50, 300),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.2),
        "max_depth": trial.suggest_int("max_depth", 5, 15),
        "num_leaves": trial.suggest_int("num_leaves", 20, 100),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 20),
        "min_split_gain": trial.suggest_float("min_split_gain", 0.0, 0.3),
        "subsample": trial.suggest_float("subsample", 0.6, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.6, 1.0),
    }


    lightGBM = LGBMRegressor(**params)
    model = MultiOutputRegressor(lightGBM)
    model.fit(X_train, y_train)

    preds = model.predict(X_valid)
    return mean_absolute_error(y_valid, preds)

# 3. Run Optuna study
study = optuna.create_study(direction="minimize")
study.optimize(lightGBM_objective, n_trials=30, show_progress_bar=True)

# 4. Train final model with best params
best_params = study.best_trial.params
best_model = MultiOutputRegressor(LGBMRegressor(**best_params))
best_model.fit(X_train, y_train)

# 5. Predict and evaluate
y_pred = best_model.predict(X_valid)
mae = mean_absolute_error(y_valid, y_pred)

# Convert y_valid and y_pred to DataFrames if they aren't already, using target column names
if not isinstance(y_valid, pd.DataFrame):
    y_valid = pd.DataFrame(y_valid, columns=target_columns)
if not isinstance(y_pred, pd.DataFrame):
    y_pred = pd.DataFrame(y_pred, columns=target_columns)

# Calculate MAE for each column
print(f"✅ Best Overall MAE: {mae:.4f}\n")
print("MAE per target column:")
for col in target_columns:
    col_mae = mean_absolute_error(y_valid[col], y_pred[col])
    print(f"{col}: {col_mae:.4f}")

[I 2025-07-31 01:24:38,826] A new study created in memory with name: no-name-3ccbe4d4-5671-46c7-95b8-7b551117e7a1
  0%|          | 0/30 [00:00<?, ?it/s]

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000238 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 514
[LightGBM] [Info] Number of data points in the train set: 265, number of used features: 11
[LightGBM] [Info] Start training from score 43.498113
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, b

Best trial: 0. Best value: 628.76:   3%|▎         | 1/30 [00:00<00:25,  1.16it/s]

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

Best trial: 0. Best value: 628.76:   7%|▋         | 2/30 [00:02<00:36,  1.30s/it]

...

Best trial: 25. Best value: 405.844:  97%|█████████▋| 29/30 [00:34<00:01,  1.12s/it]

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

Best trial: 25. Best value: 405.844: 100%|██████████| 30/30 [00:36<00:00,  1.20s/it]

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f